In [1]:
from helpers import load_all_datasets, compose, str_rep_vdb, str_rep_no_steps, CBRDatabase
import pandas as pd

DATABASE_NAME = "cbr_database"
OUTPUT_FILE = "small_validation_cbr_00_input.csv"

In [2]:
_, _, _, _, cbr_database, _, validation = load_all_datasets()
cbr_database_id = cbr_database.set_index("id")

In [3]:
cbr_database_reps = str_rep_vdb(cbr_database)
len(cbr_database_reps)

10627

In [4]:
meta = CBRDatabase.create_metadatas([int(e) for e in list(cbr_database["id"].values)], {"dataset":"cbr_database"})

# vdb = CBRDatabase(cbr_database_reps, metadatas=meta, db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=True)
vdb = CBRDatabase(db_directory=f"dataset_vector/{DATABASE_NAME}", initialize_new=False)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [5]:


# [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query="xd", k=4)]

In [6]:
# generowanie inputów - 1500 tokenów 
import random
from transformers import LlamaTokenizerFast

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")

inputs = []
pe_recipes = []
target_recipe = []
lens = []

for i, v_recipe in validation.iterrows():
    while True:
        # recipe_ixs = [random.randint(0, len(cbr_database)-1) for _ in range(7)]
        recipe_ixs = [e.metadata[CBRDatabase.RECIPE_ID_KEY] for e in vdb.similarity_search(query=str_rep_no_steps(v_recipe), k=7)]

        gen_input = compose(cbr_database_id.loc[recipe_ixs], v_recipe)
        
        if len(tokenizer.encode(gen_input)) <= 1550: 
            br = True
        else:
            for ii in list(range(6, -1, -1)):
                gen_input = compose(cbr_database_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+1:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1550: 
                    br = True
                    print(f"1 - {len(tokenizer.encode(gen_input))}")
                    break
            for ii in list(range(5, -1, -1)):
                gen_input = compose(cbr_database_id.loc[recipe_ixs[:ii] + recipe_ixs[ii+2:]], v_recipe)
                if len(tokenizer.encode(gen_input)) <= 1550: 
                    br = True
                    print(f"2 - {len(tokenizer.encode(gen_input))}")
                    break
        
        if not br:
            print(cbr_database_id.loc[recipe_ixs].steps.values)
            print(f"REPEAT - {len(tokenizer.encode(gen_input))}")
        else:
            lens.append(len(tokenizer.encode(gen_input)))
            break
            
        
    pe_recipes.append(recipe_ixs)
    target_recipe.append(v_recipe["id"])
    inputs.append(gen_input)

/home/maciej/anaconda3/envs/ml_lm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1 - 1292
2 - 1111
1 - 1363
2 - 1114
1 - 1388
2 - 1055
1 - 1380
2 - 1132
1 - 1361
2 - 1127
1 - 1311
2 - 1157
1 - 1373
2 - 1159
1 - 1291
2 - 1103
1 - 1424
2 - 1238
1 - 1306
2 - 1151
1 - 1454
2 - 1254
1 - 1504
2 - 1269
1 - 1419
2 - 1204
1 - 1390
2 - 1167
1 - 1368
2 - 1168


In [7]:
[e for e in lens if e > 1500]

[1508,
 1536,
 1528,
 1526,
 1538,
 1507,
 1505,
 1501,
 1527,
 1513,
 1501,
 1548,
 1502,
 1508]

In [8]:
I = 6
recipe_ixs[:I] + recipe_ixs[I+1:]

[77747, 173396, 120892, 119003, 391034, 329361]

In [9]:
df = pd.DataFrame()
df["input"] = inputs
df["pe_recipes"] = pe_recipes
df["target_recipe"] = target_recipe
df.to_csv(f"input_datasets/{OUTPUT_FILE}", index=False)

In [10]:
cbr_database_id.loc[[227451, 227451]]

,name,steps,ingredients
id,,,
227451,russian berry custard kissel,Place the berries and 4 cups of the water in a...,"fresh cranberries, water, sugar, potato starch"
227451,russian berry custard kissel,Place the berries and 4 cups of the water in a...,"fresh cranberries, water, sugar, potato starch"


In [11]:
import pandas as pd

dataset = pd.read_csv("input_datasets/small_validation_pe_input.csv")
for idx, row in dataset.iterrows():
    print(idx) #63233

FileNotFoundError: [Errno 2] No such file or directory: 'input_datasets/small_validation_pe_input.csv'